In [25]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns',50)

import matplotlib.pyplot as plt
import seaborn as sns

In [26]:
train=pd.read_csv('/kaggle/input/daconcard/open/train.csv')
test=pd.read_csv('/kaggle/input/daconcard/open/test.csv')
submit=pd.read_csv('/kaggle/input/daconcard/open/sample_submission.csv')

In [27]:
def datainfo(df):
    return pd.DataFrame([(col,df[col].dtype,df[col].isna().sum(),df[col].nunique(),df[col].unique()[:5]) for col in df.columns],
                       columns=['name','dtype','missing','nunique','values :5'])

In [28]:
datainfo(train)

,name,dtype,missing,nunique,values :5
0,index,int64,0,26457,"[0, 1, 2, 3, 4]"
1,gender,object,0,2,"[F, M]"
2,car,object,0,2,"[N, Y]"
3,reality,object,0,2,"[N, Y]"
4,child_num,int64,0,9,"[0, 1, 2, 3, 4]"
5,income_total,float64,0,249,"[202500.0, 247500.0, 450000.0, 157500.0, 27000..."
6,income_type,object,0,5,"[Commercial associate, Working, State servant,..."
7,edu_type,object,0,5,"[Higher education, Secondary / secondary speci..."
8,family_type,object,0,5,"[Married, Civil marriage, Separated, Single / ..."
9,house_type,object,0,6,"[Municipal apartment, House / apartment, With ..."


In [29]:
## ordinal : edu_type(Lower,Secondary,Incomplete,Higher,academ),child_num() income(Student,)
## binary : gender,car,reality,work_phone,phone,email,
## nominal : 

In [30]:
datainfo(test)

,name,dtype,missing,nunique,values :5
0,index,int64,0,10000,"[26457, 26458, 26459, 26460, 26461]"
1,gender,object,0,2,"[M, F]"
2,car,object,0,2,"[Y, N]"
3,reality,object,0,2,"[N, Y]"
4,child_num,int64,0,6,"[0, 1, 2, 3, 5]"
5,income_total,float64,0,208,"[112500.0, 135000.0, 69372.0, 225000.0, 126000.0]"
6,income_type,object,0,5,"[Pensioner, State servant, Working, Commercial..."
7,edu_type,object,0,5,"[Secondary / secondary special, Higher educati..."
8,family_type,object,0,5,"[Civil marriage, Married, Single / not married..."
9,house_type,object,0,6,"[House / apartment, With parents, Rented apart..."


In [31]:
datainfo(submit)

,name,dtype,missing,nunique,values :5
0,index,int64,0,10000,"[26457, 26458, 26459, 26460, 26461]"
1,0,int64,0,1,[0]
2,1,int64,0,1,[0]
3,2,int64,0,1,[0]


In [32]:
## 제거 
train=train.drop(['FLAG_MOBIL','index'],axis=1)
test=test.drop(['FLAG_MOBIL','index'],axis=1)

## 중복 확인 및 제거 -> 안하는게 성능 굳.
# cols=['gender', 'car', 'reality', 'child_num', 'income_total',
#       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
#       'DAYS_EMPLOYED','work_phone', 'phone', 'email',
#       'occyp_type', 'family_size','begin_month']

## 마지막에 입력된 데이터가 바뀌어도 진짜라고 가정. drop 안했을 때 성능 굳
# train=train.drop_duplicates(subset=cols,keep='last')

## 고용일 처리 -> 양수 성능 굳 ? 
# train['DAYS_BIRTH']=train['DAYS_BIRTH']*(-1/365)
# test['DAYS_BIRTH']=test['DAYS_BIRTH']*(-1/365)

# train['DAYS_EMPLOYED']=train['DAYS_EMPLOYED']*(-1/365)
# train.loc[train['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999
# test['DAYS_EMPLOYED']=test['DAYS_EMPLOYED']*(-1/365)
# test.loc[test['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999

## 가족수 이상치 나이대로 판별 이상 판단.
train=train[(train['family_size']!=20)&(train['family_size']!=15)]
# 결측값 
# train['occyp_type']=train['occyp_type'].fillna('unknown')
# test['occyp_type']=test['occyp_type'].fillna('unknown')

train.fillna('NAN', inplace=True)
train['occyp_type'].loc[(train.occyp_type == 'NAN')&(train.DAYS_EMPLOYED > 0)]='Unemployed'
train['occyp_type'].loc[(train.occyp_type == 'NAN')&(train.DAYS_EMPLOYED < 0)]='Missing'

test.fillna('NAN', inplace=True)
test['occyp_type'].loc[(test.occyp_type == 'NAN')&(test.DAYS_EMPLOYED > 0)]='Unemployed'
test['occyp_type'].loc[(test.occyp_type == 'NAN')&(test.DAYS_EMPLOYED < 0)]='Missing'

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-cop

In [33]:
## 발급수 파생변수 추가 20210511

cols=['gender', 'car', 'reality', 'child_num', 'income_total',
      'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
      'DAYS_EMPLOYED','work_phone', 'phone', 'email',
      'occyp_type', 'family_size']

train['count']=1
test['count']=1

tr_grp_count=train.groupby(cols,as_index=False).agg({'count':[('count','count')]})
ts_grp_count=test.groupby(cols,as_index=False).agg({'count':[('count','count')]})

In [34]:
colnames=[]
for col,_ in tr_grp_count.columns:
    colnames.append(col)
    
tr_grp_count.columns=colnames
ts_grp_count.columns=colnames

train=pd.merge(train,tr_grp_count,on=cols,how='left')
test=pd.merge(test,ts_grp_count,on=cols,how='left')

In [39]:
train=train.drop('count_x',axis=1)
train=train.rename(columns={'count_y':'count'})
test=test.drop('count_x',axis=1)
test=test.rename(columns={'count_y':'count'})

In [41]:
## train drop first x 20210511
# cols=['gender', 'car', 'reality', 'child_num', 'income_total',
#       'income_type', 'edu_type', 'family_type', 'house_type', 'DAYS_BIRTH',
#       'DAYS_EMPLOYED','work_phone', 'phone', 'email',
#       'occyp_type', 'family_size']

# tr_idx=train.drop_duplicates(cols,keep='first').index
# train=train.drop(tr_idx,axis=0)

In [42]:
# train['DAYS_BIRTH']=train['DAYS_BIRTH']*(-1/365)
# test['DAYS_BIRTH']=test['DAYS_BIRTH']*(-1/365)

# train['DAYS_BIRTH']=pd.cut(train['DAYS_BIRTH'],bins=[20,30,40,50,60,70],labels=['20s','30s','40s','50s','60s'])

# train['DAYS_EMPLOYED']=train['DAYS_EMPLOYED']*(-1/365)
# # train.loc[train['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999
# test['DAYS_EMPLOYED']=test['DAYS_EMPLOYED']*(-1/365)
# # test.loc[test['DAYS_EMPLOYED']<0,'DAYS_EMPLOYED']=-999

# train['DAYS_EMPLOYED']=pd.cut(train['DAYS_EMPLOYED'],bins=[-10000,0,1,5,10,20,10000],labels=['nan','1y','5y','10y','20y','>20y'])

In [43]:
# birth_order = {
#     '<20' : 0, 
#     '20s' : 1, 
#     '30s' : 2,
#     '40s' : 3, 
#     '50s' : 4 ,
#     '60s':5,
# }
# train.DAYS_BIRTH = train.DAYS_BIRTH.map(birth_order)
# test.DAYS_BIRTH = test.DAYS_BIRTH.map(birth_order)

In [44]:
# emp_order = {
#     'nan' : 0, 
#     '1y' : 1, 
#     '5y' : 2,
#     '10y' : 3, 
#     '20y' : 4,
#     '>20y':5
# }
# train.DAYS_EMPLOYED = train.DAYS_EMPLOYED.map(emp_order)
# test.DAYS_EMPLOYED = test.DAYS_EMPLOYED.map(emp_order)


In [45]:
# edu_order = {
#     'Lower secondary' : 0, 
#     'Secondary / secondary special' : 1, 
#     'Incomplete higher' : 2,
#     'Higher education' : 3, 
#     'Academic degree' : 4 
# }
# train.edu_type = train.edu_type.map(edu_order)
# test.edu_type = test.edu_type.map(edu_order)

# # train.groupby('income_type')['income_total'].median()

# income_order={
#     'Pensioner':0,
#     'Working':1,
#     'Student':2,
#     'State servant':3,
#     'Commercial associate':4
# }
# train.income_type=train.income_type.map(income_order)
# test.income_type=test.income_type.map(income_order)

# # train.groupby('family_type')['income_total'].median()

# family_order={
#     'Civil marriage':0,
#     'Separated':0,
#     'Married':1,
#     'Widow':1,
#     'Single / not married':2
# }
# train.family_type=train.family_type.map(family_order)
# test.family_type=test.family_type.map(family_order)

# # train.groupby('house_type')['income_total'].median()

# house_order={
#     'House / apartment':0,
#     'Municipal apartment':0,
#     'With parents':0,
#     'Rented apartment':1,
#     'Office apartment':1,
#     'Co-op apartment':2
# }
# train.house_type=train.house_type.map(house_order)
# test.house_type=test.house_type.map(house_order)

# # train.groupby('occyp_type')['income_total'].median()
# ## co-op 는 가격의 차이가 있다는 가정.

# occyp_order={
#     'unknown':0,
#     'Low-skill Laborers':1,
#     'Secretaries':2,
#     'Cleaning staff':2,
#     'Cooking staff':2,
#     'HR staff':3,
#     'Laborers':3,
#     'Medicine staff':3,
#     'Sales staff':3,
#     'Security staff':3,
#     'Waiters/barmen staff':3,
#     'Private service staff':4,
#     'High skill tech staff':4,
#     'Accountants':4,
#     'Core staff':4,
#     'IT staff':4,
#     'Realty agents':5,
#     'Drivers':5,
#     'Managers':5
# }
# train.occyp_type=train.occyp_type.map(occyp_order)
# test.occyp_type=test.occyp_type.map(occyp_order)

# train['gender']=train['gender'].apply(lambda x:1 if x=='M' else 0)
# train['car']=train['car'].apply(lambda x:1 if x=='N' else 0)
# train['reality']=train['reality'].apply(lambda x:1 if x=='N' else 0)

# test['gender']=test['gender'].apply(lambda x:1 if x=='M' else 0)
# test['car']=test['car'].apply(lambda x:1 if x=='N' else 0)
# test['reality']=test['reality'].apply(lambda x:1 if x=='N' else 0)

In [46]:
# train['begin_month']*=(-1/12)
# test['begin_month']*=(-1/12)

# train['begin_month']=pd.cut(train['begin_month'],bins=[-1,0.5,1.1,2.1,3.1,4.1,5.1],labels=['0y','1/2y','1y','2y','3y','4y'])
# test['begin_month']=pd.cut(test['begin_month'],bins=[-1,0.5,1.1,2.1,3.1,4.1,5.1],labels=['0y','1/2y','1y','2y','3y','4y'])

In [47]:
# begin_order={
#     '0y':0,
#     '1/2y':1,
#     '1y':2,
#     '2y':3,
#     '3y':4,
#     '4y':5
# }
# train.begin_month=train.begin_month.map(begin_order)
# test.begin_month=test.begin_month.map(begin_order)

# train['begin_order']=train['begin_month'].astype(int)
# test['begin_order']=test['begin_month'].astype(int)

In [48]:
## 시도 파생변수 : 성능 더 떨어진다.

# train['family_size']=pd.Categorical(train['family_size'],ordered=True)
# train['child_num']=pd.Categorical(train['child_num'],ordered=True)
# train['begin_month']=pd.Categorical(train['begin_month'],ordered=True)

# test['family_size']=pd.Categorical(test['family_size'],ordered=True)
# test['child_num']=pd.Categorical(test['child_num'],ordered=True)
# test['begin_month']=pd.Categorical(test['begin_month'],ordered=True)

In [49]:
## income과 gender에 따라 차이가 없다.
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="gender",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## car reality에 따라 income의 차이는 있다.
# sns.catplot(
#     data=train, kind="bar",
#     x="car", y="income_total", hue="reality",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## car reality도 차이없다. 
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="reality",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

## income_total은 제거 해본다.
# sns.catplot(
#     data=train, kind="bar",
#     x="credit", y="income_total", hue="begin_month",
#     ci="sd", palette="dark", alpha=.6, height=6
# )

In [50]:
## #했을때 비슷.
# train['gender']=train['gender'].apply(lambda x:1 if x=='M' else 0)
# train['car']=train['car'].apply(lambda x:1 if x=='N' else 0)
# train['reality']=train['reality'].apply(lambda x:1 if x=='N' else 0)
# train['work_phone']=train['work_phone'].apply(lambda x:1 if x==0 else 0)
# train['phone']=train['phone'].apply(lambda x:1 if x==0 else 0)
# train['email']=train['email'].apply(lambda x:1 if x==0 else 0)

# test['gender']=test['gender'].apply(lambda x:1 if x=='M' else 0)
# test['car']=test['car'].apply(lambda x:1 if x=='N' else 0)
# test['reality']=test['reality'].apply(lambda x:1 if x=='N' else 0)
# test['work_phone']=test['work_phone'].apply(lambda x:1 if x==0 else 0)
# test['phone']=test['phone'].apply(lambda x:1 if x==0 else 0)
# test['email']=test['email'].apply(lambda x:1 if x==0 else 0)

In [51]:
# tr_cat_dum=pd.get_dummies(train[['income_type','edu_type','family_type','house_type','occyp_type']],drop_first=True)
# ts_cat_dum=pd.get_dummies(test[['income_type','edu_type','family_type','house_type','occyp_type']],drop_first=True)

In [52]:
# train=train.drop(['income_type','edu_type','family_type','house_type','occyp_type'],axis=1)
# test=test.drop(['income_type','edu_type','family_type','house_type','occyp_type'],axis=1)

In [53]:
# train=pd.concat([train,tr_cat_dum],axis=1)
# test=pd.concat([test,ts_cat_dum],axis=1)

In [54]:
## label
## 수가 많아서 이걸로.

# from category_encoders.ordinal import OrdinalEncoder

# oe=OrdinalEncoder(handle_missing=-999)

# oe.fit(train['family#begin'])
# train['family#begin']=oe.transform(train['family#begin'])
# test['family#begin']=oe.transform(test['family#begin'])

In [55]:
## 가설1. 다중공선성
## 가설2. income_total 구분.
# train=train.drop(['DAYS_BIRTH','child_num'],axis=1)
# test=test.drop(['DAYS_BIRTH','child_num'],axis=1)

# train['income_total']=pd.cut(train['income_total'],bins=[0,100000,350000,500000,750000,1000000,2000000],labels=['F','E','D','C','B','A'])
# test['income_total']=pd.cut(test['income_total'],bins=[0,100000,350000,500000,750000,1000000,2000000],labels=['F','E','D','C','B','A'])\

In [56]:
## feature importance에는 좋다고 나오는데 성능은 1.2 뭐가 문제일까.
## 1. + - 실험. 과적합
## 2. * 실험 뭔가 맞춰지는. 다른것과 시너지가 있는것 같다.
# train['child*family']=train.apply(lambda row:row['child_num']*row['family_size'],axis=1)
# test['child*family']=test.apply(lambda row:row['child_num']*row['family_size'],axis=1)

# train=train.drop(['family_size','child_num'],axis=1)
# test=test.drop(['family_size','child_num'],axis=1)

In [57]:
# cat_cols=['gender','car','reality','income_type','edu_type','family_type',
#           'house_type','work_phone','phone','email','occyp_type','child*family',
#           'DAYS_EMPLOYED','DAYS_BIRTH']

# from itertools import combinations as comb

# for a,b in comb(cat_cols,2):
#     df=train.groupby([a,b,'begin_month'],as_index=False)['income_total'].mean()
#     df.columns=[a,b,'begin_month',f'{a}_{b}_grpmean']
#     train=pd.merge(train,df,how='left',on=[a,b,'begin_month'])
#     test=pd.merge(test,df,how='left',on=[a,b,'begin_month'])

In [58]:
# drop_cols=[]
# for col in test.columns:
#     if test[col].isnull().sum():
#         drop_cols.append(col)

In [59]:
# train=train.drop(drop_cols,axis=1)
# test=test.drop(drop_cols,axis=1)

In [60]:
train=pd.get_dummies(train,drop_first=True)
test=pd.get_dummies(test,drop_first=True)

In [61]:
train.shape,test.shape

((26453, 50), (10000, 49))

In [62]:
# train['begin_month']=train['begin_month'].astype(int)

In [63]:
X=train.drop('credit',axis=1)
y=train['credit']

from sklearn.preprocessing import normalize

X=normalize(X,axis=1)
testc=test.copy()
testc=normalize(testc,axis=1)

In [64]:
from sklearn.model_selection import train_test_split

X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=71,stratify=y)

## xgb

In [65]:
import xgboost as xgb

params={
    'objective':'multi:softprob',
    'random_state':71,
    'n_estimators':1000
}
xgb_model=xgb.XGBClassifier(**params)

xgb_model.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_test,y_test)],
              eval_metric='mlogloss',verbose=True,
              early_stopping_rounds=10)

[0]	validation_0-mlogloss:0.97266	validation_1-mlogloss:0.97695


/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[1]	validation_0-mlogloss:0.89947	validation_1-mlogloss:0.90629
[2]	validation_0-mlogloss:0.85394	validation_1-mlogloss:0.86406
[3]	validation_0-mlogloss:0.82409	validation_1-mlogloss:0.83773
[4]	validation_0-mlogloss:0.80363	validation_1-mlogloss:0.82114
[5]	validation_0-mlogloss:0.78923	validation_1-mlogloss:0.81091
[6]	validation_0-mlogloss:0.77885	validation_1-mlogloss:0.80300
[7]	validation_0-mlogloss:0.77144	validation_1-mlogloss:0.79814
[8]	validation_0-mlogloss:0.76529	validation_1-mlogloss:0.79468
[9]	validation_0-mlogloss:0.76070	validation_1-mlogloss:0.79308
[10]	validation_0-mlogloss:0.75674	validation_1-mlogloss:0.79126
[11]	validation_0-mlogloss:0.75372	validation_1-mlogloss:0.78985
[12]	validation_0-mlogloss:0.75036	validation_1-mlogloss:0.78840
[13]	validation_0-mlogloss:0.74549	validation_1-mlogloss:0.78685
[14]	validation_0-mlogloss:0.74301	validation_1-mlogloss:0.78614
[15]	validation_0-mlogloss:0.73639	validation_1-mlogloss:0.78375
[16]	validation_0-mlogloss:0.73263

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=1000, n_jobs=4, num_parallel_tree=1,
              objective='multi:softprob', random_state=71, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=None, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [66]:
xgb_pred=pd.DataFrame(xgb_model.predict_proba(testc))
xgb_pred['index']=submit['index']
xgb_pred=xgb_pred[['index',0,1,2]]
xgb_pred.head()

/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


,index,0,1,2
0,26457,0.105818,0.207735,0.686448
1,26458,0.361630,0.214214,0.424156
2,26459,0.051753,0.091378,0.856869
3,26460,0.120335,0.097700,0.781965
4,26461,0.121454,0.178253,0.700293


In [67]:
# xgb.plot_importance(model,max_num_features=10)

In [68]:
# xgb.to_graphviz(model)

In [69]:
# pred.to_csv('20210508_xgboost_all_cat_grpincome.csv',index=False)

## rf

In [70]:
from sklearn.ensemble import RandomForestClassifier

params={
    'n_estimators':1000,
    'random_state':71,
    'criterion':'gini',
    'verbose':1,
    'class_weight':'balanced',
    'n_jobs':-1,
    'oob_score':True
}
rf_model=RandomForestClassifier(**params)

rf_model.fit(X,y.values)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    1.3s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:   23.5s
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed:   29.4s finished


RandomForestClassifier(class_weight='balanced', n_estimators=1000, n_jobs=-1,
                       oob_score=True, random_state=71, verbose=1)

In [71]:
rf_pred=pd.DataFrame(rf_model.predict_proba(testc))
rf_pred['index']=submit['index']
rf_pred=rf_pred[['index',0,1,2]]
rf_pred.head()
# from sklearn.metrics import confusion_matrix

# confusion_matrix(y_test,pred)

# fi = pd.DataFrame({'feature': list(train.drop('credit',axis=1).columns),
#                    'importance': model.feature_importances_}).\
#                     sort_values('importance', ascending = False)

# fi.head()

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.3s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    1.0s
[Parallel(n_jobs=4)]: Done 1000 out of 1000 | elapsed:    1.2s finished


,index,0,1,2
0,26457,0.090496,0.159889,0.749615
1,26458,0.583963,0.193000,0.223037
2,26459,0.020318,0.034404,0.945279
3,26460,0.028387,0.062197,0.909416
4,26461,0.089419,0.183572,0.727009


In [72]:
# rf_pred.to_csv('20210510_rf.csv',index=False)

## lgb

In [73]:
import lightgbm as lgb

params={
    'objective':'multiclass',
    'random_state':71,
    'n_estimators':1000
}
lgb_model=lgb.LGBMClassifier(**params)

lgb_model.fit(X_train,y_train,
              eval_set=[(X_train, y_train),(X_test,y_test)],
              eval_metric='multi_logloss',verbose=True,
              early_stopping_rounds=10)

[1]	training's multi_logloss: 0.862629	valid_1's multi_logloss: 0.863758
Training until validation scores don't improve for 10 rounds
[2]	training's multi_logloss: 0.847398	valid_1's multi_logloss: 0.850168
[3]	training's multi_logloss: 0.8357	valid_1's multi_logloss: 0.839879
[4]	training's multi_logloss: 0.825534	valid_1's multi_logloss: 0.831843
[5]	training's multi_logloss: 0.81736	valid_1's multi_logloss: 0.824967
[6]	training's multi_logloss: 0.809943	valid_1's multi_logloss: 0.81933
[7]	training's multi_logloss: 0.803022	valid_1's multi_logloss: 0.814355
[8]	training's multi_logloss: 0.797087	valid_1's multi_logloss: 0.810275
[9]	training's multi_logloss: 0.791884	valid_1's multi_logloss: 0.80691
[10]	training's multi_logloss: 0.787077	valid_1's multi_logloss: 0.803546
[11]	training's multi_logloss: 0.782427	valid_1's multi_logloss: 0.800358
[12]	training's multi_logloss: 0.778234	valid_1's multi_logloss: 0.797693
[13]	training's multi_logloss: 0.774504	valid_1's multi_logloss: 

LGBMClassifier(n_estimators=1000, objective='multiclass', random_state=71)

In [74]:
lgb_pred=pd.DataFrame(lgb_model.predict_proba(testc))
lgb_pred['index']=submit['index']
lgb_pred=lgb_pred[['index',0,1,2]]
lgb_pred.head()

,index,0,1,2
0,26457,0.088507,0.289620,0.621872
1,26458,0.321531,0.171406,0.507063
2,26459,0.047231,0.061340,0.891430
3,26460,0.057855,0.114074,0.828071
4,26461,0.025804,0.088684,0.885512


## svm

In [75]:
X=train.drop('credit',axis=1)
y=train['credit']

from sklearn.preprocessing import StandardScaler

sd=StandardScaler()

X=sd.fit_transform(X)
testc=test.copy()
testc=sd.transform(testc)

In [76]:
from sklearn.svm import SVC

params={
    'gamma':'auto',
    'probability':True,
    'random_state':71,
#     'max_iter':300,
    'class_weight':'balanced',
    'kernel':'poly',
    'shrinking':True,
    'verbose':True
}

svc_model=SVC(**params)
svc_model.fit(X,y.values)

[LibSVM]

SVC(class_weight='balanced', gamma='auto', kernel='poly', probability=True,
    random_state=71, verbose=True)

In [77]:
svc_pred=pd.DataFrame(svc_model.predict_proba(testc))
svc_pred['index']=submit['index']
svc_pred=svc_pred[['index',0,1,2]]
svc_pred.head()

,index,0,1,2
0,26457,0.086262,0.086680,0.827058
1,26458,0.192828,0.132606,0.674567
2,26459,0.127024,0.192656,0.680320
3,26460,0.107909,0.127172,0.764919
4,26461,0.093302,0.314897,0.591801


## all

In [78]:
submit[['0','1','2']]=(xgb_pred[[0,1,2]]*rf_pred[[0,1,2]]*lgb_pred[[0,1,2]]*svc_pred[[0,1,2]])**(1/4)
submit.head()

,index,0,1,2
0,26457,0.092469,0.169929,0.717250
1,26458,0.338268,0.175085,0.424128
2,26459,0.050116,0.078071,0.837180
3,26460,0.067956,0.096897,0.819235
4,26461,0.071508,0.173866,0.718700


In [79]:
submit.to_csv('20210511_rf*xgb*lgb*svc_occpy+count.csv',index=False)

In [ ]:
## income_total 정규성 x -> log 변환 한것을 파생변수로.
## 다른건 몰라도 income_type이 pensioner인 사람들이 DAYS_BIRTH가 오래됐다. 
## -> DAYS_BIRTH는 -7500정도에서 -25000정도 사이 나이로 바꿀 필요가 있을까? -> 보류
## edu_type에 따라 income_total과 DAYS_BIRTH는 선형관계가 있다. 
## DAYS_EMPLOYED 양수 값은 고용되지 않은 상태를 의미함 -> 제거 
## family type과 성별에 따라 income_total의 차이가 없다. 
## FLAG_MOBIL 1밖에 없다. -> 제거 
## 중복값 제거
## family_size child_num 상관관계 -> 제거,income_total income_total_log 제거 그 외 그냥 둔다.

sns.jointplot(data=train, x="family_size", y="begin_month", hue="house_type")
# sns.displot(data=train, x="income_total", col="DAYS_BIRTH", kde=True)
# sns.relplot(data=train,x='income_total',y='DAYS_BIRTH',hue='edu_tpe')

In [ ]:
## 연속형 - 연속
sns.relplot(
    data=train,
    x="income_total", y="tip", col="time",
    hue="smoker", style="smoker", size="size",
)

sns.lmplot(data=tips, x="total_bill", y="tip", col="time", hue="smoker")
sns.displot(data=tips, x="total_bill", col="time", kde=True)
sns.jointplot(data=penguins, x="flipper_length_mm", y="bill_length_mm", hue="species")

In [ ]:
sns.catplot(data=train[train['income_total']>=],kind='violin',x='credit',y='income_total')

In [ ]:
sns.catplot(data=train, kind="violin", x="gender", y="income_total_log", hue="family_type",split=False)

In [ ]:
## 범주 - 연속
sns.catplot(data=tips, kind="swarm", x="day", y="total_bill", hue="smoker",split=True)

## torch

In [ ]:
# import torch
# from torch.autograd import Variable
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler

In [ ]:
# train=pd.get_dummies(train,drop_first=True)
# test=pd.get_dummies(test,drop_first=True)

In [ ]:
# train.shape,test.shape

In [ ]:
# X=train.drop('credit',axis=1)
# y=train['credit']

# from sklearn.preprocessing import normalize

# X=normalize(X,axis=1)
# testc=test.copy()
# testc=normalize(testc,axis=1)

In [ ]:
# from sklearn.model_selection import train_test_split

# X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.15,random_state=71,stratify=y)

In [ ]:
# import random

# def set_seed(seed,mode=None):
#     torch.manual_seed(seed)
#     torch.backends.cudnn.deterministic=True
#     torch.backends.cudnn.benchmark=False
#     np.random.seed(seed)
#     random.seed(seed)
#     if mode=='reproductibility':
#         torch.cuda.manual_seed(seed)
#         torch.cuda.manual_seed_all(seed)
        
# set_seed(71)

# # def seed_torch(seed=42):
# #     random.seed(seed)
# #     os.environ['PYTHONHASHSEED'] = str(seed)
# #     np.random.seed(seed)
# #     torch.manual_seed(seed)
# #     torch.cuda.manual_seed(seed)
# #     torch.backends.cudnn.deterministic = True

# # seed_torch(seed=seed_value)

In [ ]:
# class CustomDataset(Dataset):
#     def __init__(self,X_data,y_data=None,train=True):
#         self.X_data=X_data
#         if train:
#             self.y_data=y_data
        
#     def __len__(self):
#         return len(self.X_data)
    
#     def __getitem__(self,index):
#         if train:
#             return self.X_data[index],self.y_data[index]
#         else:
#             return self.X_data[index]
    
# train_dataset=CustomDataset(torch.from_numpy(X_train).float(),torch.from_numpy(y_train.values).long())
# val_dataset=CustomDataset(torch.from_numpy(X_test).float(),torch.from_numpy(y_test.values).long())
# # test_dataset=CustomDataset(torch.from_numpy(testc).float())

In [ ]:
# target_list = []
# for _, t in train_dataset:
#     target_list.append(t)
    
# target_list = torch.tensor(target_list)
# target_list = target_list[torch.randperm(len(target_list))]

In [ ]:
# def get_class_distribution(obj):
#     count_dict = {
#         "0": 0,
#         "1": 0,
#         "2": 0
#     }
    
#     for i in obj:
#         if i == 0: 
#             count_dict['0'] += 1
#         elif i == 1: 
#             count_dict['1'] += 1
#         elif i == 2: 
#             count_dict['2'] += 1            
#         else:
#             print("Check classes.")
            
#     return count_dict

# class_count = [i for i in get_class_distribution(y_train).values()]
# class_weights = 1./torch.tensor(class_count, dtype=torch.float) 

In [ ]:
class_weights_all = class_weights[target_list]

In [ ]:
# weighted_sampler = WeightedRandomSampler(
#     weights=class_weights_all,
#     num_samples=len(class_weights_all),
#     replacement=True
# )

In [ ]:
# EPOCHS = 10
# BATCH_SIZE = 16
# LEARNING_RATE = 0.0007
# NUM_FEATURES = 47
# NUM_CLASSES = 3

# device='cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
# train_loader=DataLoader(dataset=train_dataset,
#                           batch_size=BATCH_SIZE,
#                           sampler=weighted_sampler
# )
# val_loader=DataLoader(dataset=val_dataset,batch_size=1)
# test_loader=DataLoader(dataset=test_dataset,batch_size=1)

In [ ]:
# class MCNet(nn.Module):
#     def __init__(self,num_feature,num_class):
#         super(MCNet,self).__init__()
#         self.layer_1 = nn.Linear(num_feature, 512)
#         self.layer_2 = nn.Linear(512, 128)
#         self.layer_3 = nn.Linear(128, 64)
#         self.layer_out = nn.Linear(64, num_class) 
        
#         self.relu = nn.ReLU()
# #         self.dropout = nn.Dropout(p=0.2)
# #         self.batchnorm1 = nn.BatchNorm1d(512)
# #         self.batchnorm2 = nn.BatchNorm1d(128)
# #         self.batchnorm3 = nn.BatchNorm1d(64)
        
#     def forward(self, x):
#         x = self.layer_1(x)
# #         x = self.batchnorm1(x)
#         x = self.relu(x)
        
#         x = self.layer_2(x)
# #         x = self.batchnorm2(x)
#         x = self.relu(x)
# #         x = self.dropout(x)
        
#         x = self.layer_3(x)
# #         x = self.batchnorm3(x)
#         x = self.relu(x)
# #         x = self.dropout(x)
        
#         x = self.layer_out(x)
        
#         return x

In [ ]:
# model = MCNet(num_feature = NUM_FEATURES, num_class=NUM_CLASSES)
# model.to(device)

# criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
# optimizer = optim.Adam(model.parameters(),lr=LEARNING_RATE)

In [ ]:
# !pip install torchsummary

In [ ]:
# from torchsummary import summary

# summary(model,(1,47))

In [ ]:
# def multi_acc(y_pred, y_test):
#     y_pred_softmax = torch.log_softmax(y_pred, dim = 1)
#     y_pred_tags = torch.max(y_pred_softmax, dim = 1)[1]
    
#     correct_pred = (y_pred_tags == y_test).float()
#     acc = correct_pred.sum() / len(correct_pred)
    
#     acc = torch.round(acc * 100)
    
#     return acc

In [ ]:
# accuracy_stats = {
#     'train': [],
#     "val": []
# }
# loss_stats = {
#     'train': [],
#     "val": []
# }

In [ ]:
# batch_size=128
# n_iters=5000
# num_epochs=n_iters/128
# num_epochs=int(num_epochs)

# # Traning the Model
# count=0
# loss_list=[]
# iteration_list=[]
# accuracy_list=[]

# for epoch in range(num_epochs):
#     for i,(train_x,train_y) in enumerate(train_loader):
        
#         # Define variables
#         train_x=Variable(train_x).to(device)
#         train_y=Variable(train_y).to(device)
        
#         # Clear gradients
#         optimizer.zero_grad()

#         # Forward propagation
#         outputs=model(train_x)
        
#         # Calculate softmax and cross entropy loss
#         loss=criterion(outputs,train_y)
        
#         # Calculate gradients
#         loss.backward()
        
#         # Update parameters
#         optimizer.step()
        
#         count+=1
        
#         # validation
#         if count%50==0:
#             # Calculate Accuracy         
#             correct=0
#             total=0
#             # Predict test dataset
#             for val_x,val_y in val_loader: 
#                 val=Variable(val_x).to(device)
#                 labels=Variable(val_y).to(device)
                
#                 # Forward propagation
#                 outputs=model(val)
                
#                 # Get predictions from the maximum value
#                 predicted=torch.max(outputs.data,1)[1]
                
#                 # Total number of labels
# #                 total+=len(labels)
                
#                 # Total correct predictions
# #                 correct+=(predicted==labels).sum()
            
# #             accuracy=100*correct/float(total)
            
#             # store loss and iteration
#             loss_list.append(loss.data)
#             iteration_list.append(count)
# #             accuracy_list.append(accuracy)
#         if count%500==0:
#             # Print Loss
#             print(f'Iteration: {count}  Loss: {loss.data}')

In [ ]:
# print("Begin training.")
# for e in range(1, EPOCHS+1):
    
#     # TRAINING
#     train_epoch_loss = 0
#     train_epoch_acc = 0
#     model.train()
#     for X_train_batch, y_train_batch in train_loader:
#         X_train_batch, y_train_batch = X_train_batch.to(device), y_train_batch.to(device)
#         optimizer.zero_grad()
        
#         y_train_pred = model(X_train_batch)
        
#         train_loss = criterion(y_train_pred, y_train_batch)
#         train_loss = Variable(train_loss, requires_grad = True)
#         train_acc = multi_acc(y_train_pred, y_train_batch)
        
#         train_loss.backward()
#         optimizer.step()
        
#         train_epoch_loss += train_loss.item()
#         train_epoch_acc += train_acc.item()
        
        
#     # VALIDATION    
#     with torch.no_grad():
        
#         val_epoch_loss = 0
#         val_epoch_acc = 0
        
#         model.eval()
#         for X_val_batch, y_val_batch in val_loader:
#             X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            
#             y_val_pred = model(X_val_batch)
                        
#             val_loss = criterion(y_val_pred, y_val_batch)
#             val_acc = multi_acc(y_val_pred, y_val_batch)
            
#             val_epoch_loss += val_loss.item()
#             val_epoch_acc += val_acc.item()
#     loss_stats['train'].append(train_epoch_loss/len(train_loader))
#     loss_stats['val'].append(val_epoch_loss/len(val_loader))
#     accuracy_stats['train'].append(train_epoch_acc/len(train_loader))
#     accuracy_stats['val'].append(val_epoch_acc/len(val_loader))
                              
    
#     print(f'Epoch {e+0:03}: | Train Loss: {train_epoch_loss/len(train_loader):.5f} | Val Loss: {val_epoch_loss/len(val_loader):.5f} | Train Acc: {train_epoch_acc/len(train_loader):.3f}| Val Acc: {val_epoch_acc/len(val_loader):.3f}')

In [ ]:
# with torch.no_grad():
#     model.eval()
#     data=torch.from_numpy(testc).float()
#     res=model(data)
#     preds=torch.max(res,1)[1]
#     preds = [a.squeeze().tolist() for a in preds]